In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim

# Load our dataset
df = pd.read_csv("data.csv")

# Drop the unnecessary columns
df = df.drop(columns=['StudentID', 'GradeClass'])

# Separate features from target
X = df.drop(columns=["GPA"]) # Features
y = df['GPA'] # Target

# Normalize the columns of the dataset
scaler = StandardScaler() # Mean = 0, Standard Deviation = 1
X_scaled = scaler.fit_transform(X)  # Fit the scaler to the features and transform them

# Convert to tensors for use with PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32) # Convert features to tensor
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1) # Convert target to tensor

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=123)

# Create DataLoader objects for training and testing data
train_data = TensorDataset(X_train, y_train) # Pair features and target
test_data = TensorDataset(X_test, y_test)  # Pair features and target

# DataLoader objects are used to automatically create batches of data
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the neural network architecture
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # First hidden layer with 64 neurons
        self.fc2 = nn.Linear(64, 32)         # Second hidden layer with 32 neurons
        self.fc3 = nn.Linear(32, 1)          # Output layer (predict GPA)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation function after first layer
        x = torch.relu(self.fc2(x))  # Apply ReLU activation function after second layer
        x = self.fc3(x)              # Output layer (no activation, since we're predicting a continuous value)
        return x

# Initialize the model with the number of features
model = MLP(input_dim=X_train.shape[1])

# Define the loss function
loss_fn = nn.MSELoss()  # Mean Squared Error for regression

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(X_batch)  # Forward pass
        loss = loss_fn(outputs, y_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        running_loss += loss.item() * X_batch.size(0)
    
    avg_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Evaluate the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    y_pred = model(X_test)
    test_loss = loss_fn(y_pred, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")

Epoch [1/100], Loss: 1.5852
Epoch [2/100], Loss: 0.1809
Epoch [3/100], Loss: 0.0849
Epoch [4/100], Loss: 0.0724
Epoch [5/100], Loss: 0.0649
Epoch [6/100], Loss: 0.0595
Epoch [7/100], Loss: 0.0547
Epoch [8/100], Loss: 0.0518
Epoch [9/100], Loss: 0.0490
Epoch [10/100], Loss: 0.0473
Epoch [11/100], Loss: 0.0452
Epoch [12/100], Loss: 0.0439
Epoch [13/100], Loss: 0.0422
Epoch [14/100], Loss: 0.0413
Epoch [15/100], Loss: 0.0401
Epoch [16/100], Loss: 0.0404
Epoch [17/100], Loss: 0.0387
Epoch [18/100], Loss: 0.0383
Epoch [19/100], Loss: 0.0371
Epoch [20/100], Loss: 0.0369
Epoch [21/100], Loss: 0.0356
Epoch [22/100], Loss: 0.0355
Epoch [23/100], Loss: 0.0355
Epoch [24/100], Loss: 0.0352
Epoch [25/100], Loss: 0.0340
Epoch [26/100], Loss: 0.0336
Epoch [27/100], Loss: 0.0331
Epoch [28/100], Loss: 0.0328
Epoch [29/100], Loss: 0.0325
Epoch [30/100], Loss: 0.0321
Epoch [31/100], Loss: 0.0316
Epoch [32/100], Loss: 0.0317
Epoch [33/100], Loss: 0.0320
Epoch [34/100], Loss: 0.0314
Epoch [35/100], Loss: 0